In [1]:
from keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 2s 0us/step


In [2]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape,  y_test.shape)

(50000, 100) (10000, 100)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=2045)

print(X_train.shape, y_train.shape, '/', X_valid.shape, y_valid.shape, '/', X_test.shape, y_test.shape)

(35000, 32, 32, 3) (35000, 100) / (15000, 32, 32, 3) (15000, 100) / (10000, 32, 32, 3) (10000, 100)


In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=1000)
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=1000)
test_generator = test_datagen.flow(X_test, y_test, batch_size=1000)

In [5]:
from keras.applications import VGG16

conv_base = VGG16(weights= 'imagenet', include_top = False, input_shape=(32,32,3))
conv_base.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

In [8]:
set_trainble = False

for layer in conv_base.layers:
  if layer.name == 'block4_conv3':
    set_trainble = True

  if set_trainble:
    layer.trainable = True
  else:
    layer.trainable = False

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [11]:
from keras import models, layers, optimizers

model = models.Sequential()
model.add(conv_base)

model.add(layers.Dense(518, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.000005), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 1, 1, 518)         265734    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 518)         0         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1, 100)         51900     
Total params: 15,032,322
Trainable params: 9,756,866
Non-trainable params: 5,275,456
_________________________________________________________________
